Análisis de sentimientos de mensajes en Amazon usando Naive Bayes
===

El archivo que se encuentra disponible en el link

https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/amazon_cells_labelled.tsv

contiene mensajes escritos por los usuarios para productos comprados en Amazon y su valoración (positiva, negativa e indeterminada). En este laboratorio se debe construir un clasificador bayesiano que debe ser entrenado con los mensajes valorados, el cual debe ser posteriormente utilizado para valorar los mensajes con valoración indeterminada.

##**Carga de datos y de librerias**

In [32]:
import warnings
import pandas as pd
import pytest
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn import svm
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error



warnings.filterwarnings("ignore")

Los datos son `tsv`, se brinda nombre a las columnas y los datos faltantes se remplazan con NA, la cual hace referencia a una valuación indeterminada.

In [33]:
pd.set_option("display.notebook_repr_html", True)

data = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/amazon_cells_labelled.tsv",
    sep="\t",
    header=None)

data.columns=["Text","label"]



data

,Text,label
0,I try not to adjust the volume setting to avoi...,NaN
1,So there is no way for me to plug it in here i...,0.0
2,"Good case, Excellent value.",1.0
3,I thought Motorola made reliable products!.,NaN
4,Battery for Motorola Razr.,NaN
...,...,...
14604,"The screen on my phone said ""Not Charging"".",NaN
14605,This is my 4th Samsung cell phone with T-Mobile.,NaN
14606,great company.,NaN
14607,"The ""call"" and ""hang-up"" keys are now properly...",NaN


In [34]:
#
# Cargue el archivo usando Pandas e imprima la cantidad de
# registros
#
# Rta/
# 14609
#
import pandas as pd

data.shape[0]

14609

In [35]:
#
# Imprima el primer mensaje de texto.
#
# Rta/
# 'I try not to adjust the volume setting to avoid that I turn off the call button which is situated just below the volume adjustment knob.'
#
data.iloc[0][0]

'I try not to adjust the volume setting to avoid that I turn off the call button which is situated just below the volume adjustment knob.'

In [36]:
#
# Imprima la cantidad de mensajes con NaN
#
# Rta/
# 13609
#
data['label'].isna().sum()



13609

In [37]:
#
# Imprima la cantidad de mensajes con valoración igual a 1.0
#
# Rta/
# 500
#
filtr = data['label'] == 1
filtr.sum()

500

In [38]:
#
# Imprima la cantidad de mensajes con valoración igual a 0.0
#
# Rta/
# 500
#
filtr = data['label'] == 0
filtr.sum()

500

In [39]:
#
# Genere un nuevo dataset que contenga únicamente los registros
# con valoración positiva o negativa e imprima su longitud
#
# Rta/
# 1000
#

new_data = data.copy()
new_data = new_data.dropna()
new_data.shape[0]



1000

In [40]:
#
# Genere una nueva columna en el nuevo dataset computada como
# el resultado de aplicar el stemmer de Porter al mensaje e
# imprima el primer mensaje transformado
#
# Rta/
# 'so there is no way for me to plug it in here in the us unless i go by a converter.'
#

from nltk.stem import PorterStemmer 

ps = PorterStemmer()

#new_data["PorterStemmer"] = new_data["Text"].str.lower().apply(lambda x: [ps.stem(y) for y in x])
#new_data.iloc[0,2]
#new_data


#new_data['Splitted']=new_data['Text'].str.lower().apply(lambda x : filter(None,x.split(" ")))
#new_data['ApplyPorter']=new_data['Splitted'].apply(lambda x : [ps.stem(y) for y in x])
#new_data['PorterStemmer']=new_data['ApplyPorter'].apply(lambda x : " ".join(x))
new_data['PorterStemmer']=new_data['Text'].apply(lambda x: " ".join([ps.stem(w) for w in x.split()])).str.lower()
data_Porter=new_data[['Text', 'PorterStemmer','label']]
data_Porter.iloc[0,1]

'so there is no way for me to plug it in here in the us unless i go by a converter.'

In [41]:
#
# Construya la matriz de terminos del documento considerando
# las palabras que tengan una frecuencia entre el 0.1% y el 98%,
# y que esten unicamente conformadas por letras.
#
# Imprima el tamaño del vocabulario.
#
# Rta/
# 1554
#

from sklearn.feature_extraction.text import CountVectorizer

#token_pattern=r'(?u)\b\w*[a-zA-Z]\w*\b'

#vectorizer = CountVectorizer(max_df=0.98, min_df=0.0001,stop_words='english')
#vectorizer = CountVectorizer(analyzer="word",max_df=0.98, min_df=0.0001,token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b")
vectorizer= CountVectorizer(
    analyzer="word",                # a nivel de palabra
    lowercase=True,                 # convierte a minúsculas
    stop_words="english",           # stop_words en inglés
    token_pattern=r"(?u)\b\w\w+\b", # patrones a reconocer
    max_df=0.98,                     # máxima frecuencia a considerar
    min_df=1,                       # ignora palabras con baja frecuencia
)

X = vectorizer.fit_transform(data_Porter['PorterStemmer']).toarray()
df_new = pd.DataFrame(X, columns=vectorizer.get_feature_names())

df_new.shape[1]


1554

In [42]:
#
# Construya un clasificador bayesiano que use los primeros
# 500 patrones para entrenamiento y los últimos 500 para
# prueba, e imprima el porcentaje de datos para cada clase
# para la muestra de entrenamiento-
#
# Rta/
# 1.0    52.2
# 0.0    47.8
# Name: label, dtype: float64
#
X_train = df_new[0:500]
X_test = df_new[500:]

y_train = new_data.label[0:500]
y_test = new_data.label[500:]

round(100 * y_train.value_counts() / sum(y_train.value_counts()), 1)

1.0    52.2
0.0    47.8
Name: label, dtype: float64

In [43]:
#
# Imprima el porcentaje de datos para cada clase para la muestra
# de prueba, redondeado a un decimal.
#
# Rta/
# 0.0    52.2
# 1.0    47.8
# Name: label, dtype: float64
#

round(100 * y_test.value_counts() / sum(y_train.value_counts()), 1)


0.0    52.2
1.0    47.8
Name: label, dtype: float64

In [44]:
#
# Cree un clasificador de Bayes y entrenelo. Realice el pronostico
# para la muestra de entrenamiento y compute la matriz de confusion
#
# Rta/
# array([[214,  25],
#        [  1, 260]])
#
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

gnb = BernoulliNB()

gnb.fit(X_train, y_train)

y_train_pred = gnb.predict(X_train)


r = confusion_matrix(y_true=y_train, y_pred=y_train_pred)
r.astype('int')

array([[214,  25],
       [  1, 260]])

In [45]:
#
# Realice el pronóstico para la muestra de entrenamiento y compute
# la matriz de confusión
#
# Rta/
# array([[153, 108],
#        [ 32, 207]])
#

y_test_pred = gnb.predict(X_test)


r = confusion_matrix(y_true=y_test, y_pred=y_test_pred)
r.astype('int')

array([[153, 108],
       [ 32, 207]])

In [46]:
#
# Realice el pronostico para los mensajes con valoración 
# indeterminada y compute la cantidad de mensajes positivos
#
# Rta/
# 8284
#
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer

ps = PorterStemmer()

Data_Predicted = data[data.isnull().any(axis=1)]

#apply PorterStemmer

Data_Predicted['PorterStemmer']=Data_Predicted['Text'].apply(lambda x: " ".join([ps.stem(w) for w in x.split()])).str.lower()
Data_Predicted=Data_Predicted[['Text', 'PorterStemmer','label']]

X = vectorizer.transform(Data_Predicted.PorterStemmer)

Pronostico = gnb.predict(X)
Positive = list(filter(lambda x: x==1,Pronostico))
len(Positive)

8284